# Feature Engineering: Latent Dirichlet Allocation

## imports

In [4]:
import pickle
import numpy as np
import pandas as pd
from datetime import date
import json
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import re
from collections import Counter, defaultdict
import itertools
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize, regexp_tokenize
import gensim
from gensim.corpora.dictionary import Dictionary
import spacy

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import warnings
warnings.filterwarnings('ignore')

In [6]:
sw = stopwords.words("english")

In [7]:
with open(f'../data/metascripts_df_profanity.pickle', 'rb') as file:
    metascripts = pickle.load(file)

## prepare the data

In [8]:
descriptions = list(metascripts['description'].values)
scripts = list(metascripts['transcript'].values)
scripts_dict = dict(zip(descriptions, scripts))

## tokenize and lemmatize

In [25]:
nlp = spacy.load('en_core_web_md')

def chunker(iterable, chunksize):
    for i in range(0, len(list(iterable)), chunksize):
        yield iterable[i:i+chunksize]

chunksize = 2
junk =  ['\n', '♪']
corpus = []
for scripts_subset in tqdm(chunker(scripts, 2), total = np.ceil(len(scripts)/chunksize)):
    corpus.extend([[token.lemma_ for token in script if token.text not in junk and not token.is_punct and not token.is_stop] for script in nlp.pipe(scripts_subset)])

  0%|          | 0/155.0 [00:00<?, ?it/s]

## CountVectorizer and TfidfVectorizer

In [29]:
def dummy(doc):
    return doc

ct_vectorizer = CountVectorizer(lowercase = True, 
                             tokenizer = dummy,
                             preprocessor = dummy
                            )
scripts_tf = ct_vectorizer.fit_transform(corpus)

tfidf_vectorizer = TfidfVectorizer(**ct_vectorizer.get_params())
scripts_tfidf = tfidf_vectorizer.fit_transform(corpus)

In [30]:
tfidf_vectorizer.get_feature_names_out()[8000:8015]

array(['Kaufman', 'Kavanaugh', 'Kavi', 'Kavin', 'Kawasaki', 'Kay', 'Kaye',
       'Kaz', 'Kazan', 'Ke$ha', 'Keanu', 'Keats', 'Keegan', 'Keema',
       'Keeno'], dtype=object)

In [33]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=50, random_state=0)
lda_tf.fit(scripts_tf)
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=50, random_state=0)
lda_tfidf.fit(scripts_tfidf)

LatentDirichletAllocation(n_components=50, random_state=0)

In [36]:
pyLDAvis.sklearn.prepare(lda_tf, scripts_tf, ct_vectorizer, mds='mmds')

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
35     0.015321  0.205218       1        1  36.319762
36     0.035956  0.366661       2        1  14.594112
24     0.027789  0.295143       3        1  13.800967
13     0.076291  0.388403       4        1   5.079756
41     0.161715  0.362132       5        1   4.051591
22     0.066685  0.290303       6        1   3.672994
2      0.001360  0.280397       7        1   3.620522
11    -0.001696  0.318920       8        1   3.434084
32    -0.057183  0.307277       9        1   3.429648
14     0.117420  0.316503      10        1   2.228628
21     0.161635  0.276420      11        1   1.316646
29    -0.190028  0.288767      12        1   1.224274
45    -0.094825  0.322802      13        1   1.200711
37    -0.288370  0.276217      14        1   1.150526
27     0.283729  0.203474      15        1   0.812604
34    -0.149955  0.210353      16        1   0.694548
47    -0.229156  0.143167      17        1   0.483000
46    -0.237776  0.105819      18        1   0.428881
25     0.221942  0.124502      19        1   0.418984
42     0.124315  0.160149      20        1   0.394308
12    -0.314793  0.061095      21        1   0.386027
33     0.315875  0.014488      22        1   0.298249
9      0.240368  0.056868      23        1   0.285781
23    -0.328631 -0.052226      24        1   0.262456
19    -0.287616 -0.125842      25        1   0.186755
17     0.281630 -0.176780      26        1   0.156684
28     0.133691 -0.246629      27        1   0.055413
39    -0.003954 -0.207742      28        1   0.000606
38    -0.003715 -0.207539      29        1   0.000522
43    -0.003715 -0.207539      30        1   0.000522
44    -0.003715 -0.207539      31        1   0.000522
48    -0.003715 -0.207539      32        1   0.000522
40    -0.003715 -0.207539      33        1   0.000522
0     -0.003715 -0.207539      34        1   0.000522
31    -0.003715 -0.207539      35        1   0.000522
30    -0.003715 -0.207539      36        1   0.000522
26    -0.003715 -0.207539      37        1   0.000522
1     -0.003715 -0.207539      38        1   0.000522
20    -0.003715 -0.207539      39        1   0.000522
18    -0.003715 -0.207539      40        1   0.000522
16    -0.003715 -0.207539      41        1   0.000522
15    -0.003715 -0.207539      42        1   0.000522
10    -0.003715 -0.207539      43        1   0.000522
8     -0.003715 -0.207539      44        1   0.000522
7     -0.003715 -0.207539      45        1   0.000522
6     -0.003715 -0.207539      46        1   0.000522
5     -0.003715 -0.207539      47        1   0.000522
4     -0.003715 -0.207539      48        1   0.000522
3     -0.003715 -0.207539      49        1   0.000522
49    -0.003715 -0.207539      50        1   0.000522, topic_info=           Term          Freq         Total Category  logprob  loglift
9                17851.000000  17851.000000  Default    30.00  30.0000
27154      like  43731.000000  43731.000000  Default    29.00  29.0000
39317             5420.000000   5420.000000  Default    28.00  28.0000
26693      know  24387.000000  24387.000000  Default    27.00  27.0000
23876        go  13242.000000  13242.000000  Default    26.00  26.0000
...         ...           ...           ...      ...      ...      ...
25         "Did      0.000158      1.471830  Topic50   -10.58   3.0210
26     "Whereas      0.000158      1.232135  Topic50   -10.58   3.1988
27        "whee      0.000158      1.489149  Topic50   -10.58   3.0093
28       "would      0.000158      1.456892  Topic50   -10.58   3.0312
29        "yeah      0.000158      1.471830  Topic50   -10.58   3.0210

[3522 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
0          1  0.372373    \n\n
0          3  0.434436    \n\n
0         27  0.155156    \n\n
1          1  0.406990  \n\n\n
1          3  0.406990  \n\n\n
...      ...       ...     ...
39322     2

In [34]:
pyLDAvis.sklearn.prepare(lda_tfidf, scripts_tfidf, tfidf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
22    -0.333812 -0.012200       1        1  54.760704
24     0.055456 -0.148994       2        1  13.232566
35     0.036844  0.068728       3        1   7.729365
5      0.028765  0.035042       4        1   5.479739
33     0.020693  0.019316       5        1   3.518516
8      0.012776  0.009377       6        1   1.881346
27     0.006654  0.003064       7        1   0.777490
29     0.004984  0.001423       8        1   0.520501
26     0.004892  0.001363       9        1   0.461353
41     0.004862  0.001375      10        1   0.445915
25     0.004731  0.001164      11        1   0.418298
47     0.004551  0.001042      12        1   0.410716
0      0.004660  0.001198      13        1   0.401367
20     0.004519  0.001043      14        1   0.395137
46     0.004580  0.001116      15        1   0.392829
16     0.004399  0.000943      16        1   0.380345
9      0.004440  0.000973      17        1   0.376574
36     0.004358  0.000874      18        1   0.362246
13     0.004252  0.000751      19        1   0.341928
38     0.004247  0.000779      20        1   0.335505
45     0.004203  0.000755      21        1   0.325101
21     0.004201  0.000759      22        1   0.324849
31     0.004140  0.000772      23        1   0.312892
34     0.004059  0.000645      24        1   0.304637
28     0.003984  0.000579      25        1   0.288675
40     0.003971  0.000586      26        1   0.287682
39     0.003905  0.000497      27        1   0.272422
42     0.003914  0.000501      28        1   0.270305
4      0.003830  0.000435      29        1   0.265774
23     0.003823  0.000415      30        1   0.260041
30     0.003854  0.000454      31        1   0.258794
2      0.003827  0.000402      32        1   0.253024
15     0.003811  0.000426      33        1   0.252414
3      0.003764  0.000376      34        1   0.249943
14     0.003732  0.000330      35        1   0.242706
19     0.003765  0.000342      36        1   0.238639
49     0.003725  0.000285      37        1   0.238060
7      0.003720  0.000357      38        1   0.236949
18     0.003708  0.000347      39        1   0.232653
48     0.003725  0.000371      40        1   0.231760
32     0.003653  0.000304      41        1   0.222515
37     0.003629  0.000268      42        1   0.218559
44     0.003564  0.000216      43        1   0.217372
17     0.003614  0.000258      44        1   0.216375
11     0.003670  0.000301      45        1   0.214653
6      0.003602  0.000237      46        1   0.213743
12     0.003532  0.000187      47        1   0.203087
43     0.003501  0.000146      48        1   0.188942
10     0.003475  0.000142      49        1   0.186148
1      0.003247 -0.000070      50        1   0.148844, topic_info=               Term       Freq      Total Category  logprob  loglift
27154          like  85.000000  85.000000  Default  30.0000  30.0000
26693          know  50.000000  50.000000  Default  29.0000  29.0000
9                    47.000000  47.000000  Default  28.0000  28.0000
23876            go  27.000000  27.000000  Default  27.0000  27.0000
32884         right  25.000000  25.000000  Default  26.0000  26.0000
...             ...        ...        ...      ...      ...      ...
34856         spank   0.000587   0.070267  Topic50  -9.5811   1.7243
30482    pedophilia   0.000535   0.067512  Topic50  -9.6735   1.6718
10790  Philadelphia   0.000581   0.074643  Topic50  -9.5914   1.6536
19417  contribution   0.000515   0.066497  Topic50  -9.7108   1.6497
26248      jamaican   0.000496   0.066478  Topic50  -9.7494   1.6114

[1566 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
9          1  1.009237        
36         1  1.006403  'cause
2720       4  1.715133    Bill
4504       2  0.831508     Dad
7742       2  1.076640   Jesus
...      ...       ...     ...
38917      1  1.024338   woman
3902